In [1]:
from dotenv import load_dotenv
load_dotenv()


In [5]:
from langchain.prompts import PromptTemplate
from langchain.document_loaders import YoutubeLoader
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [15]:
def create_db(url):
    loader=YoutubeLoader(url,language=['en','en-US'])
    transcript=loader.load()
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    docs=text_splitter.split_documents(transcript)
    embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    db=FAISS.from_documents(docs,embeddings)
    return db

In [18]:
def create_chain():
    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        You are a helpful assistant that that can answer questions about youtube videos
        based on the video's transcript.

        Answer the following question: {question}
        By searching the following video transcript: {docs}

        Only use the factual information from the transcript to answer the question.

        If you feel like you don't have enough information to answer the question, say "I don't know".

        Your answers should be verbose and detailed.
        """)
    
    llm=ChatGoogleGenerativeAI(model='gemini-pro',
                           temperature=0.7,
                           top_p=0.85)
    
    chain=LLMChain(llm=llm,prompt=prompt)
    return chain

In [9]:
def run_chain(query,chain,db):
    docs=db.similarity_search(query,10)
    docs_page_content=" ".join([d.page_content for d in docs])

    response=chain.run(question=query,docs=docs_page_content)
    return response

In [16]:
url="Vxw0nE1qfZc"
db=create_db(url)

In [19]:
chain=create_chain()

In [20]:
run_chain(query="What does video talk about?",db=db,chain=chain)

'This video talks about the differences between data science and AI engineering. Data scientists are described as data storytellers who use mathematical models to translate messy real world data into insights, while AI engineers are described as AI system builders who use foundation models to build generative AI systems that help to transform business processes. The video also discusses the different use cases, data types, models, and processes that are used in each field.'